In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']

# Install on Google Colab
import subprocess
import sys

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["ampform", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# Inspect model interactively

In this notebook, we illustrate how to interactively inspect the {class}`.HelicityModel` that was created in {doc}`amplitude`. We do this with the excellent {doc}`mpl-interactions <mpl_interactions:index>` package.

First, use {mod}`pickle` to {func}`~pickle.load` the {class}`.HelicityModel` that we created in {doc}`amplitude`:

In [ ]:
import pickle

with open("helicity_model.pickle", "rb") as stream:
    model = pickle.load(stream)

In this case, {ref}`as we saw <usage/amplitude:Mathematical formula>`, the overall model contains just one intensity term $I = |\sum_i A_i|^2$, with $\sum_i A_i$ some coherently sum of amplitudes. We can get the amplitude $\sum_i A_i$ as follows:

In [ ]:
import sympy as sp

amplitude = model.expression.args[0].args[0].args[0]
assert isinstance(amplitude, sp.Add)

Replace some of the boring parameters with the provided {attr}`~.HelicityModel.parameter_defaults`:

In [ ]:
amplitude = amplitude.doit().subs(
    {
        s: v
        for s, v in model.parameter_defaults.items()
        if not s.name.startswith("Gamma") and not s.name.startswith("m_f(0)")
    }
)
symbols = sorted(amplitude.free_symbols, key=lambda s: s.name)
symbols

Identify the symbols:

In [ ]:
(
    gamma_f1500,
    gamma_f980,
    m_1,
    m,
    m_2,
    m_f1500,
    m_f980,
    phi,
    theta,
) = tuple(symbols)

...and use them to {func}`~sympy.utilities.lambdify.lambdify` the expression:

In [ ]:
np_amplitude = sp.lambdify(
    (m, m_f980, m_f1500, gamma_f980, gamma_f1500, m_1, m_2, phi, theta),
    amplitude,
    "numpy",
)

Now we are ready to use {doc}`mpl-interactions <mpl_interactions:index>` to investigate the behavior of the amplitude model. First, we define some functions that formulate what we want to plot:

In [ ]:
import numpy as np


def intensity(
    m_values, m_f980, m_f1500, gamma_f980, gamma_f1500, m_pi, phi, theta
):
    values = np_amplitude(
        m_values,
        m_f980,
        m_f1500,
        gamma_f980,
        gamma_f1500,
        m_pi,
        m_pi,
        phi,
        theta,
    )
    return np.abs(values) ** 2


def argand(m_f980, m_f1500, gamma_f980, gamma_f1500, m_pi, phi, theta):
    values = np_amplitude(
        m_values,
        m_f980,
        m_f1500,
        gamma_f980,
        gamma_f1500,
        m_pi,
        m_pi,
        phi,
        theta,
    )
    argand = np.array([values.real, values.imag])
    return argand.T


def imag(m_values, m_f980, m_f1500, gamma_f980, gamma_f1500, m_pi, phi, theta):
    values = np_amplitude(
        m_values,
        m_f980,
        m_f1500,
        gamma_f980,
        gamma_f1500,
        m_pi,
        m_pi,
        phi,
        theta,
    )
    return values.imag


def real(m_values, m_f980, m_f1500, gamma_f980, gamma_f1500, m_pi, phi, theta):
    values = np_amplitude(
        m_values,
        m_f980,
        m_f1500,
        gamma_f980,
        gamma_f1500,
        m_pi,
        m_pi,
        phi,
        theta,
    )
    return values.real

Next, we need to define the domain over which to plot, as well as a range/domain for the sliders that are to represent the parameter values:

In [ ]:
m_values = np.linspace(0.2, 2.5, num=400)
m_f980_values = np.linspace(0.8, 1.8, num=100)
m_f1500_values = np.linspace(0.8, 1.8, num=100)
gamma_f980_values = np.linspace(0.01, 1, num=100)
gamma_f1500_values = np.linspace(0.01, 1, num=100)
m_pi_values = np.linspace(0.01, 1, 10)
phi_values = np.arange(0, 2 * np.pi, step=np.pi / 40)
theta_values = np.arange(-np.pi, np.pi, step=np.pi / 40)

Finally, use {doc}`mpl-interactions <mpl_interactions:index>` to plot the functions defined above with regard to the parameter values:

:::{margin}

Interactive {mod}`~matplotlib.widgets` do not render well on web pages, so run this notebook in Google Colab or in Jupyter Lab to see the full power of {doc}`mpl-interactions <mpl_interactions:index>`!

:::

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)
if STATIC_WEB_PAGE:
    m_f980_values = np.linspace(0.8, 2.2, 100)
    m_f980_values = np.concatenate((m_f980_values, np.flip(m_f980_values[1:])))

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt

fig, axes = plt.subplots(
    2, 2, figsize=0.9 * np.array((8, 7)), tight_layout=True
)
fig.suptitle(R"$J/\psi \to \gamma f_0, f_0 \to \pi^0\pi^0$")
((ax_intensity, ax_argand), (ax_real, ax_imag)) = axes
m_label = R"$m_{\pi^0\pi^0}$ (GeV)"
for ax in axes.flat:
    if ax is ax_argand:
        continue
    ax.set_xlabel(m_label)
ax_argand.set_xlabel("Re($A$)")
ax_argand.set_ylabel("Im($A$)")
ax_real.set_ylabel("Re($A$)")
ax_imag.set_ylabel("Im($A$)")
ax_intensity.set_ylabel("$I = |A|^2$")

controls = iplt.plot(
    m_values,
    intensity,
    m_f980=m_f980_values,
    m_f1500=m_f1500_values,
    gamma_f980=gamma_f980_values,
    gamma_f1500=gamma_f1500_values,
    phi=phi_values,
    theta=theta_values,
    m_pi=m_pi_values,
    ylim="auto",
    ax=ax_intensity,
    slider_formats={
        "m_f980": "{:.3f}",
        "m_f1500": "{:.3f}",
        "m_f1500": "{:.3f}",
        "gamma_f980": "{:.3f}",
        "gamma_f1500": "{:.3f}",
        "phi": "{:.3f}",
        "theta": "{:.3f}",
        "m_pi": "{:.3f}",
    },
)
iplt.scatter(
    argand,
    controls=controls,
    xlim="auto",
    ylim="auto",
    parametric=True,
    c=m_values,
    s=1,
    ax=ax_argand,
)
iplt.plot(m_values, real, controls=controls, ylim="auto", ax=ax_real)
iplt.plot(m_values, imag, controls=controls, ylim="auto", ax=ax_imag)

plt.colorbar(label=m_label, ax=ax_argand, aspect=30, pad=0.01)
plt.winter()

Helper for resetting the parameter values:

In [ ]:
def find_index(array, value):
    for i in range(len(array)):
        if array[i] >= value:
            return i
    return 0


def set_value(name, values, start_value):
    controls.controls[name].children[0].value = find_index(values, start_value)

In [ ]:
%%capture
import qrules as q

pdg = q.load_pdg()
set_value("m_f980", m_f980_values, pdg["f(0)(980)"].mass)
set_value("gamma_f980", gamma_f980_values, pdg["f(0)(980)"].width)
set_value("m_f1500", m_f1500_values, pdg["f(0)(1500)"].mass)
set_value("gamma_f1500", gamma_f1500_values, pdg["f(0)(1500)"].width)
set_value("phi", phi_values, np.pi)
set_value("theta", theta_values, 0)
set_value("m_pi", m_pi_values, pdg["pi0"].mass)

In [ ]:
%%capture
# Export for Read the Docs
output_path = "animation.gif"
if STATIC_WEB_PAGE:
    controls.save_animation(output_path, fig, "m_f980", fps=25)

:::{margin}

This figure is an animation over **just one of the parameters**. Run the notebook itself to play around with all parameters!

:::

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import Image

    with open(output_path, "rb") as f:
        display(Image(data=f.read(), format="png"))